In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re
import csv
import plotly.express as px
import plotly.graph_objects as go
import math
import ast



In [2]:
selection = pd.read_csv("/media/hdd/tmp/bsp/meta.csv", dtype={'market_id': object, 'selection_id': object}, parse_dates = ['market_time'])

selection.head(3)

,market_id,selection_id,venue,market_time,selection_name,win,bsp
0,1.179845158,23493550,Cowra,2021-03-01 04:15:00,1. Larmour,0,6.20
1,1.179845158,16374800,Cowra,2021-03-01 04:15:00,3. Careering Away,1,3.60
2,1.179845158,19740699,Cowra,2021-03-01 04:15:00,4. Bells N Bows,0,6.62


In [18]:
prices = pd.read_csv(
    "/media/hdd/tmp/bsp/prices.csv", 
    quoting=csv.QUOTE_ALL,
    dtype={'market_id': 'string', 'selection_id': 'string', 'atb_ladder': 'string', 'atl_ladder': 'string'},
    parse_dates=['time']
)
prices['atb_ladder'] = [ast.literal_eval(x) for x in prices['atb_ladder']]
prices['atl_ladder'] = [ast.literal_eval(x) for x in prices['atl_ladder']]

# Drop the first row within each group
prices = prices.drop(prices.groupby(['market_id', 'selection_id'],as_index=False).nth(0).index)

prices.head(3)

,market_id,selection_id,time,traded_volume,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder
21,1.179845158,23493550,2021-03-01 04:13:00.058,2465,5.93,3.17,113,238,"{'p': [6.2, 6, 5.9, 5.8, 5.7], 'v': [14.63, 13...","{'p': [6.6, 6.8, 7, 7.2, 7.4], 'v': [22.86, 12..."
22,1.179845158,16374800,2021-03-01 04:13:00.058,5046,3.35,1.70,449,300,"{'p': [3.65, 3.6, 3.55, 3.5, 3.45], 'v': [0.45...","{'p': [3.75, 3.8, 3.9, 4.1, 4.3], 'v': [5.14, ..."
23,1.179845158,19740699,2021-03-01 04:13:00.058,1978,6.39,3.25,154,251,"{'p': [6, 5.9, 5.8, 5.7, 5.6], 'v': [4.71, 89....","{'p': [6.4, 6.6, 6.8, 7, 7.2], 'v': [30.24, 2...."


In [19]:
prices.shape

(3937805, 10)

In [20]:
prices.query('market_id == "1.183995724" and selection_id == "22832649"')

,market_id,selection_id,time,traded_volume,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder
2762714,1.183995724,22832649,2021-06-01 01:38:00.062,1894,8.60,7.69,27,184,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [31.23, 37...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [63.98, 14..."
2762724,1.183995724,22832649,2021-06-01 01:38:10.158,2082,8.60,7.69,27,184,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [25.54, 49...","{'p': [8.2, 8.4, 8.6, 8.8, 9], 'v': [37.63, 68..."
2762734,1.183995724,22832649,2021-06-01 01:38:20.159,2094,8.00,7.69,27,184,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [94.52, 49...","{'p': [8.2, 8.4, 8.6, 8.8, 9], 'v': [37.63, 56..."
2762744,1.183995724,22832649,2021-06-01 01:38:30.182,2229,8.00,7.69,27,190,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [62.88, 49...","{'p': [8.2, 8.4, 8.6, 8.8, 9], 'v': [22.99, 56..."
2762754,1.183995724,22832649,2021-06-01 01:38:40.221,2240,8.00,7.69,136,205,"{'p': [8.2, 8, 7.8, 7.6, 7.4], 'v': [13.92, 12...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [53.28, 41..."
2762764,1.183995724,22832649,2021-06-01 01:38:50.923,2294,8.00,7.69,136,205,"{'p': [8, 7.8, 7.6, 7.4, 7.2], 'v': [129.4, 53...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [67.76, 41..."
2762774,1.183995724,22832649,2021-06-01 01:39:00.955,2297,8.00,7.69,137,214,"{'p': [8.2, 8, 7.8, 7.6, 7.4], 'v': [26.93, 13...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [53.28, 41..."
2762784,1.183995724,22832649,2021-06-01 01:39:10.962,2417,8.00,7.69,137,229,"{'p': [8.2, 8, 7.8, 7.6, 7.4], 'v': [29.2, 146...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [53.26, 23..."
2762794,1.183995724,22832649,2021-06-01 01:39:20.966,2677,8.18,2.68,212,243,"{'p': [8.4, 8.2, 8, 7.8, 7.6], 'v': [19.2, 89....","{'p': [8.6, 8.8, 9, 9.2, 9.4], 'v': [69.9, 16...."
2762804,1.183995724,22832649,2021-06-01 01:39:30.971,2795,8.18,2.68,212,243,"{'p': [8.2, 8, 7.8, 7.6, 7.4], 'v': [79.45, 82...","{'p': [8.4, 8.6, 8.8, 9, 9.2], 'v': [59.89, 65..."


In [21]:
selection.query('market_id == "1.183995724" and selection_id == "22832649"')

,market_id,selection_id,venue,market_time,selection_name,win,bsp
106966,1.183995724,22832649,Hamilton,2021-06-01 01:40:00,1. Ablestock,0,8.4


In [22]:
# Define the betfair tick ladder
def bfTickLadder():

    tickIncrements = {
        1.0: 0.01,
        2.0: 0.02,
        3.0: 0.05,
        4.0: 0.1,
        6.0: 0.2,
        10.0: 0.5,
        20.0: 1.0,
        30.0: 2.0,
        50.0: 5.0,
        100.0: 10.0,
        1000.0: 1000,
    }

    ladder = []

    for index, key in enumerate(tickIncrements):

        increment = tickIncrements[key]

        if (index+1) == len(tickIncrements):
            ladder.append(key)
        else:
            key1 = [*tickIncrements][index]
            key2 = [*tickIncrements][index+1]
            steps = (key2 - key1) / increment

            for i in range(int(steps)):
                ladder.append(round(key + i * increment, 2))

    return(ladder)

bfticks = bfTickLadder()
    
# Round a decimal to the betfair tick value below
def bfTickFloor(price, includeIndex=False):

    if 'bfticks' in globals():
        global bfticks
    else:
        bfticks = bfTickLadder()

    ind = [ n for n,i in enumerate(bfticks) if i>=price][0]
    if includeIndex:
        if bfticks[ind]==price:
            return((ind, price))
        else:
            return((ind-1, bfticks[ind-1]))
    else:
        if bfticks[ind]==price:
            return(price)
        else:
            return(bfticks[ind-1])

# Calculate the numder of ticks between two tick values
def bfTickDelta(p1, p2):

    if np.isnan(p1) or np.isnan(p2):
        return(np.nan)

    x = bfTickFloor(p1, includeIndex=True)
    y = bfTickFloor(p2, includeIndex=True)
    return(x[0]-y[0])

def bfTickShift(p, rungs):

    if 'bfticks' in globals():
        global bfticks
    else:
        bfticks = bfTickLadder()
    
    flr = bfTickFloor(p, includeIndex = True)

    return(bfticks[flr[0]+rungs])


def bfLadderMidPoint(p1, p2):

    if np.isnan(p1) or np.isnan(p2):
        return(np.nan)

    delta = -1 * bfTickDelta(p1, p2)

    if delta == 1:
        return(p1)
    elif delta % 2 != 0:
        return(bfTickShift(p1, math.ceil(delta / 2)))
    else:
        return(bfTickShift(p1, math.floor(delta / 2)))

In [62]:
# Join and augment
df = (
    selection.merge(prices, on = ['market_id', 'selection_id'])
    .assign(sbsj = lambda x: round((x['market_time'] - x['time']).dt.total_seconds() / 10) * 10)
    .assign(back_best = lambda x: [np.nan if d.get('p') is None else d.get('p')[0] for d in x['atb_ladder']])
    .assign(lay_best = lambda x: [np.nan if d.get('p') is None else d.get('p')[0] for d in x['atl_ladder']])
    .assign(geometric_mid_point = lambda x: round(1 / np.sqrt((1/x['back_best']) * (1/x['lay_best'])), 3))
    .assign(ladder_mid_point = lambda x: x.apply(lambda x: bfLadderMidPoint(x.back_best, x.lay_best), axis=1))
    .replace([np.inf, -np.inf], np.nan)
)

df.head(3)


,market_id,selection_id,venue,market_time,selection_name,win,bsp,time,traded_volume,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder,sbsj,back_best,lay_best,geometric_mid_point,ladder_mid_point
0,1.179845158,23493550,Cowra,2021-03-01 04:15:00,1. Larmour,0,6.2,2021-03-01 04:13:00.058,2465,5.93,3.17,113,238,"{'p': [6.2, 6, 5.9, 5.8, 5.7], 'v': [14.63, 13...","{'p': [6.6, 6.8, 7, 7.2, 7.4], 'v': [22.86, 12...",120.0,6.2,6.6,6.397,6.4
1,1.179845158,23493550,Cowra,2021-03-01 04:15:00,1. Larmour,0,6.2,2021-03-01 04:13:10.077,2848,5.93,3.17,113,238,"{'p': [6, 5.9, 5.8, 5.7, 5.6], 'v': [59.93, 36...","{'p': [6.4, 6.6, 6.8, 7, 7.2], 'v': [28.79, 50...",110.0,6.0,6.4,6.197,6.2
2,1.179845158,23493550,Cowra,2021-03-01 04:15:00,1. Larmour,0,6.2,2021-03-01 04:13:20.161,2866,5.93,3.17,113,238,"{'p': [6.2, 6, 5.9, 5.8, 5.7], 'v': [22.91, 88...","{'p': [6.6, 6.8, 7, 7.2, 7.4], 'v': [55.19, 22...",100.0,6.2,6.6,6.397,6.4


# The BSP

- Volumes traded
- Accuracy
- Back profit / lay profit in certain odds ranges

In [13]:
# Volume Traded
# ___________________________


In [91]:
# Accuracy
# ___________________________

# Prepare dataframe with BSP and the geometric midpoint first

preplay = df[df.sbsj.isin([120,90,60,30,0])][['market_id', 'selection_id', 'win', 'sbsj', 'geometric_mid_point']].sort_values(['market_id', 'selection_id', 'sbsj'], ascending = [True, True, False]).rename(columns={'geometric_mid_point': 'odds'}).assign(type = lambda x: "seconds before off: " + x['sbsj'].astype(int).astype(str))

bsp = df.sort_values(['market_id', 'selection_id', 'time'], ascending = [True, True, False]).groupby(['market_id', 'selection_id']).head(1)[['market_id', 'selection_id', 'win', 'sbsj', 'bsp']].rename(columns={'bsp': 'odds'}).assign(type = "bsp")

accuracyFrame = pd.concat([preplay, bsp]).dropna()
accuracyFrame

,market_id,selection_id,win,sbsj,odds,type
2790008,1.171091071,10693094,0,120.0,134.164,seconds before off: 120
2790011,1.171091071,10693094,0,90.0,149.666,seconds before off: 90
2790014,1.171091071,10693094,0,60.0,200.000,seconds before off: 60
2790017,1.171091071,10693094,0,30.0,239.792,seconds before off: 30
2790020,1.171091071,10693094,0,-0.0,239.165,seconds before off: 0
...,...,...,...,...,...,...
2253017,1.184845396,24644026,0,-20.0,39.580,bsp
2253081,1.184845396,28640078,0,-20.0,13.240,bsp
2253065,1.184845396,37583926,0,-20.0,70.420,bsp
2253049,1.184845396,37996263,1,-20.0,5.970,bsp


In [92]:
# Implied chance vs Actual chance ++++++++++++++++++++++++++++++++++++++++

# BSP Only
# __________________

winRates = (
    accuracyFrame
    .query('type == "bsp"')
    .assign(implied_chance = lambda x: round(20 * (1 / x['odds']))/20)
    .groupby('implied_chance', as_index = False)
    .agg({'win': 'mean'})
)

fig = px.scatter(winRates, x = "implied_chance", y = "win", template = "plotly_white", title = "BSP: implied win vs actual win")
fig.add_trace(
    go.Scatter(
        x = winRates.implied_chance, y = winRates.implied_chance, name = 'no bias', line_color = 'rgba(8,61,119, 0.3)'
    )
)
fig.show()

In [93]:
# Odds Comparison
# __________________

winRates = (
    accuracyFrame
    .assign(implied_chance = lambda x: round(20 * (1 / x['odds']))/20)
    .groupby(['type', 'implied_chance'], as_index = False)
    .agg({'win': 'mean'})
)

fig = px.scatter(winRates, x = "implied_chance", y = "win", color = 'type', template = "plotly_white", title = "Comparing Price Points: implied win vs actual win")
fig.add_trace(
    go.Scatter(
        x = winRates.implied_chance, y = winRates.implied_chance, name = 'no bias', line_color = 'rgba(8,61,119, 0.3)'
    )
)
fig.show()


In [95]:
# Logloss ++++++++++++++++++++++++++++++++++++++++

from sklearn.metrics import log_loss

# Overall Logloss
# _________________

bspLoss = log_loss(
    y_pred = 1 / accuracyFrame.query('type == "bsp"')['odds'],
    y_true = accuracyFrame.query('type == "bsp"')['win']
)

print(f'The overall logloss of the BSP is {round(bspLoss,4)}')

The overall logloss of the BSP is 0.2757


In [96]:

# Logloss at Different Time Points
# _________________

accuracyFrame.groupby('type', as_index = False).apply(lambda x: log_loss(y_pred=1/x['odds'],y_true=x['win'])).rename(columns = {None: 'logloss'}).sort_values('logloss')

,type,logloss
0,bsp,0.275679
1,seconds before off: 0,0.275824
3,seconds before off: 30,0.275999
4,seconds before off: 60,0.276044
5,seconds before off: 90,0.276206
2,seconds before off: 120,0.276256


In [114]:
fig = px.bar(
    accuracyFrame.groupby('type', as_index = False).apply(lambda x: log_loss(y_pred=1/x['odds'],y_true=x['win'])).rename(columns = {None: 'logloss'}).sort_values('logloss', ascending = False),
    x = "type",
    y = "logloss",
    template = "plotly_white",
    title = "Logloss Of Odds At Various Time Points"
)
fig.update_yaxes(range=[.2755, .2765])
fig.show()

# Projecting BSP

- Near price
    + showing 60 second lag
    +

In [67]:
# VISUALISATION
# ________________________



# Show 60 second lag of the near price

# def chartPrices(dSlice):

dSlice = df.query('market_id == "1.182394184" and selection_id == "39243409"').dropna()

dSlice


,market_id,selection_id,venue,market_time,selection_name,win,bsp,time,traded_volume,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,atb_ladder,atl_ladder,sbsj,back_best,lay_best,geometric_mid_point,ladder_mid_point
1664988,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:03:00.064,14073,2.97,2.01,650,661,"{'p': [2.96, 2.94, 2.92, 2.9, 2.88], 'v': [24....","{'p': [2.98, 3, 3.05, 3.1, 3.15], 'v': [10.01,...",120.0,2.96,2.98,2.970,2.96
1664989,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:03:10.106,14579,2.97,2.01,664,939,"{'p': [2.96, 2.94, 2.92, 2.9, 2.88], 'v': [223...","{'p': [2.98, 3, 3.05, 3.1, 3.15], 'v': [59.52,...",110.0,2.96,2.98,2.970,2.96
1664990,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:03:20.181,15107,2.97,2.01,664,967,"{'p': [2.96, 2.94, 2.92, 2.9, 2.88], 'v': [38....","{'p': [2.98, 3, 3.05, 3.1, 3.15], 'v': [7.76, ...",100.0,2.96,2.98,2.970,2.96
1664991,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:03:30.203,15346,2.97,2.01,669,973,"{'p': [2.98, 2.96, 2.94, 2.92, 2.9], 'v': [71....","{'p': [3, 3.05, 3.1, 3.15, 3.2], 'v': [80.54, ...",90.0,2.98,3.00,2.990,2.98
1664992,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:03:40.259,17248,3.00,2.54,774,973,"{'p': [3, 2.98, 2.96, 2.94, 2.92], 'v': [34.6,...","{'p': [3.05, 3.1, 3.15, 3.2, 3.25], 'v': [1444...",80.0,3.00,3.05,3.025,3.00
1664993,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:03:50.466,19506,3.00,2.54,774,973,"{'p': [3, 2.98, 2.96, 2.94, 2.92], 'v': [14.38...","{'p': [3.05, 3.1, 3.15, 3.2, 3.25], 'v': [1438...",70.0,3.00,3.05,3.025,3.00
1664994,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:04:00.494,21456,3.00,2.54,774,973,"{'p': [2.98, 2.96, 2.94, 2.92, 2.9], 'v': [249...","{'p': [3, 3.05, 3.1, 3.15, 3.2], 'v': [334.69,...",60.0,2.98,3.00,2.990,2.98
1664995,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:04:10.565,30742,3.00,2.54,801,982,"{'p': [3.1, 3.05, 3, 2.98, 2.96], 'v': [345.54...","{'p': [3.15, 3.2, 3.25, 3.3, 3.35], 'v': [555....",50.0,3.10,3.15,3.125,3.10
1664996,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:04:20.604,33417,3.00,2.54,832,1007,"{'p': [3.05, 3, 2.98, 2.96, 2.94], 'v': [85.36...","{'p': [3.1, 3.15, 3.2, 3.25, 3.3], 'v': [5.55,...",40.0,3.05,3.10,3.075,3.05
1664997,1.182394184,39243409,Randwick,2021-04-25 05:05:00,4. Rubamos,0,2.79,2021-04-25 05:04:30.691,35432,3.00,2.54,832,1241,"{'p': [3, 2.98, 2.96, 2.94, 2.92], 'v': [48.05...","{'p': [3.05, 3.1, 3.15, 3.2, 3.25], 'v': [38.1...",30.0,3.00,3.05,3.025,3.00


In [68]:
def chartClosingPrices(d):

    fig = px.line(
        pd.melt(d[:-1][['sbsj', 'back_best', 'near_price']], id_vars = 'sbsj', var_name = 'price'), 
            x='sbsj', y='value',
            color = 'price',
            template='plotly_white',
            title="Selection",
            labels = {
                'sbsj': "Seconds Before Scheduled Jump"
            }
    )
    fig.update_layout(font_family="Roboto")
    fig.add_trace(
        go.Line(x = dSlice.sbsj, y = dSlice.bsp, name = 'BSP', line_color = 'rgba(8,61,119, 0.3)', mode = "lines")
    )
    fig['layout']['xaxis']['autorange'] = "reversed"
    fig.show()

chartClosingPrices(dSlice)


/home/tmbish/.local/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [115]:
# Measurement
# ________________________

# Comparing the accuracy of several methods - at the scheduled off

estimatesDf = df[df.sbsj == 0][['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point']]

In [116]:

(
    pd.melt(estimatesDf, id_vars = 'bsp', var_name = 'estimate')
    .assign(error = lambda x: abs(x['value'] - x['bsp']) / x['bsp'])
    .groupby('estimate', as_index=False)
    .agg({'error': 'mean'})
    .sort_values('error')
)

,estimate,error
0,back_best,0.091702
3,ladder_mid_point,0.093181
2,geometric_mid_point,0.094405
4,lay_best,0.103142
5,near_price,0.121266
1,far_price,0.578425


In [126]:
# We can see that although the near price is a perfect run of the BSP execution algorithm it's not nearly the most accurate estimator cause of the lag


# Let's isolate the last second which is the last second before the market goes inplay to show that it is indeed the best if not for the lag

lastEstimatesDf = df.groupby(['market_id', 'selection_id'],as_index=False).nth(-1)[['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point']]

(
    pd.melt(lastEstimatesDf, id_vars = 'bsp', var_name = 'estimate')
    .assign(error = lambda x: abs(x['value'] - x['bsp']) / x['bsp'])
    .groupby('estimate', as_index=False)
    .agg({'error': 'mean'})
    .sort_values('error')
)

# Ok not event he best estimate just before the off

,estimate,error
2,geometric_mid_point,0.039627
3,ladder_mid_point,0.042526
0,back_best,0.063622
5,near_price,0.077108
4,lay_best,0.098198
1,far_price,0.290777


# ML Attempt

Let's see if we can build a better estimate of the BSP than any of our existing metrics

In [219]:
ladder = df.atb_ladder.iloc[0]
print(ladder)
# stake = 200
# liability_pool = 200
# price = ladder['p']
# volume = ladder['v']
# liability = [(a-1) * b for a, b in zip(price, volume)]
# print(liability)
# cLiability = np.cumsum(liability)
# print(cLiability)

# min([ i for (i,j) in enumerate(cLiability) if j > liability_pool ])
# # indmax = min([ i for (i,j) in enumerate(cLiability) if j > liability ])+1
# # return(round(sum([a * b for a, b in zip(price[:indmax], volume[:indmax])]) / sum(volume[:indmax]),4))



def wapToGetBack(pool, ladder):
    price = ladder['p']
    volume = ladder['v']
    try:
        indmax = min([ i for (i,j) in enumerate(cVolume) if j > pool ])+1
    except:
        indmax = len(volume)
    return(round(sum([a * b for a, b in zip(price[:indmax], volume[:indmax])]) / sum(volume[:indmax]),4))

def wapToGetLay(liability_pool, ladder):
    price = ladder['p']
    volume = ladder['v']
    liability = [(a-1) * b for a, b in zip(price, volume)]
    cLiability = np.cumsum(liability)
    try:
        indmax = min([ i for (i,j) in enumerate(cLiability) if j > liability_pool ])+1
    except:
        indmax = len(volume)
    return(round(sum([a * b for a, b in zip(price[:indmax], volume[:indmax])]) / sum(volume[:indmax]),4))

wapToGetBack(2000000, ladder)

wapToGetLay(2000000, ladder)

{'p': [6.2, 6, 5.9, 5.8, 5.7], 'v': [14.63, 136.22, 29.28, 37.25, 42.16]}


5.9226

In [223]:
model_matrix = df[['sbsj', 'atb_ladder', 'atl_ladder','bsp', 'traded_volume', 'near_price', 'far_price', 'bsp_back_pool_stake', 'bsp_lay_pool_liability', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point']]

# Filter at scheduled jump
model_matrix = model_matrix[model_matrix.sbsj == 0].dropna()

# Some feature engineering
# Will add the weighted average price required to back at the BSP back pool stake
# And the weighted average price required to lay at the BSP lay pool stake

def wapToGetBack(pool, ladder):
    price = ladder['p']
    volume = ladder['v']
    try:
        indmax = min([ i for (i,j) in enumerate(cVolume) if j > pool ])+1
    except:
        indmax = len(volume)
    return(round(sum([a * b for a, b in zip(price[:indmax], volume[:indmax])]) / sum(volume[:indmax]),4))

def wapToGetLay(liability_pool, ladder):
    price = ladder['p']
    volume = ladder['v']
    liability = [(a-1) * b for a, b in zip(price, volume)]
    cLiability = np.cumsum(liability)
    try:
        indmax = min([ i for (i,j) in enumerate(cLiability) if j > liability_pool ])+1
    except:
        indmax = len(volume)
    return(round(sum([a * b for a, b in zip(price[:indmax], volume[:indmax])]) / sum(volume[:indmax]),4))

model_matrix = (
    model_matrix
    .assign(wap_to_get_back_pool = lambda x: x.apply(lambda x: wapToGetBack(x.bsp_back_pool_stake, x.atb_ladder), axis=1))
    .assign(wap_to_get_lay_pool = lambda x: x.apply(lambda x: wapToGetLay(x.bsp_lay_pool_liability, x.atl_ladder), axis=1))
)

# Convert to probabilities
for col in ['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point', 'wap_to_get_back_pool', 'wap_to_get_lay_pool']:
    model_matrix[col] = 1 / model_matrix[col]

# Drop other columns
model_matrix.drop(columns = ['sbsj', 'atb_ladder', 'atl_ladder'], inplace = True)

model_matrix.head(3)

,bsp,traded_volume,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,back_best,lay_best,geometric_mid_point,ladder_mid_point,wap_to_get_back_pool,wap_to_get_lay_pool
12,0.161290,6891,0.174216,0.236967,518,1580,0.166667,0.161290,0.163961,0.166667,0.173124,0.156910
28,0.277778,13579,0.280112,0.578035,1023,1771,0.289855,0.277778,0.283768,0.281690,0.302939,0.270219
44,0.151057,5911,0.172117,0.628931,845,1378,0.161290,0.151515,0.156323,0.156250,0.169045,0.140515


In [225]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Setup Train / Test
train_features, test_features, train_labels, test_labels = train_test_split(model_matrix.drop(columns = ['bsp']), model_matrix['bsp'], test_size = 0.25)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

# Instantiate Model
rf = RandomForestRegressor(n_estimators = 100)

# Train Model
rf.fit(train_features, train_labels)

Training Features Shape: (119822, 11)
Training Labels Shape: (119822,)
Testing Features Shape: (39941, 11)
Testing Labels Shape: (39941,)


RandomForestRegressor()

In [226]:
# Use the forest's predict method on the test data
predicted_bsp_prob = rf.predict(test_features)
predicted_bsp_prob

array([0.02467371, 0.0312447 , 0.11456683, ..., 0.00283577, 0.04084674,
       0.11241565])

In [229]:
# Let's test our estimate vs our others in the same way as before

testDf = test_features
testDf['bsp'] = test_labels
testDf['rf_bsp_prediction'] = predicted_bsp_prob

for col in ['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point', 'rf_bsp_prediction', 'wap_to_get_back_pool', 'wap_to_get_lay_pool']:
    testDf[col] = 1 / testDf[col]

testDf.head(3)


,traded_volume,near_price,far_price,bsp_back_pool_stake,bsp_lay_pool_liability,back_best,lay_best,geometric_mid_point,ladder_mid_point,wap_to_get_back_pool,wap_to_get_lay_pool,bsp,rf_bsp_prediction
1696983,803,35.34,13.19,225,2948,42.0,44.0,42.988,42.0,0.026493,0.019670,37.07,40.528965
2261008,2736,27.80,5.00,475,4627,32.0,34.0,32.985,32.0,0.034267,0.026493,31.61,32.005428
1117864,4940,8.77,1.80,366,675,8.6,8.8,8.699,8.6,0.124513,0.109905,9.40,8.728530


In [230]:
(
    pd.melt(testDf[['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point', 'rf_bsp_prediction']], id_vars = 'bsp', var_name = 'estimate')
    .assign(error = lambda x: abs(x['value'] - x['bsp']) / x['bsp'])
    .groupby('estimate', as_index=False)
    .agg({'error': 'mean'})
    .sort_values('error')
)

,estimate,error
6,rf_bsp_prediction,0.086749
0,back_best,0.090688
3,ladder_mid_point,0.092202
2,geometric_mid_point,0.093363
4,lay_best,0.102073
5,near_price,0.120600
1,far_price,0.579210


In [240]:
# Validate it on a completely different time point - 10 seconds after scheduled jump

outOfSample = df[['sbsj', 'atb_ladder', 'atl_ladder','bsp', 'traded_volume', 'near_price', 'far_price', 'bsp_back_pool_stake', 'bsp_lay_pool_liability', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point']]

outOfSample = outOfSample[outOfSample.sbsj == -10].dropna()

outOfSample = (
    outOfSample
    .assign(wap_to_get_back_pool = lambda x: x.apply(lambda x: wapToGetBack(x.bsp_back_pool_stake, x.atb_ladder), axis=1))
    .assign(wap_to_get_lay_pool = lambda x: x.apply(lambda x: wapToGetLay(x.bsp_lay_pool_liability, x.atl_ladder), axis=1))
)


modelInputs = outOfSample

# Convert to probabilities
for col in ['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point', 'wap_to_get_back_pool', 'wap_to_get_lay_pool']:
    modelInputs[col] = 1 / modelInputs[col]

# Drop other columns
modelInputs.drop(columns = ['sbsj', 'atb_ladder', 'atl_ladder'], inplace = True)

outofsamplebspprediction = rf.predict(modelInputs.drop(columns = 'bsp'))

outofsamplebspprediction


array([0.15982679, 0.28651998, 0.15968522, ..., 0.00227314, 0.00492101,
       0.01890296])

In [241]:
outOfSample['rf_bsp_prediction'] = outofsamplebspprediction

for col in ['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point', 'rf_bsp_prediction', 'wap_to_get_back_pool', 'wap_to_get_lay_pool']:
    outOfSample[col] = 1 / outOfSample[col]

(
    pd.melt(outOfSample[['bsp', 'near_price', 'far_price', 'back_best', 'lay_best', 'geometric_mid_point', 'ladder_mid_point', 'rf_bsp_prediction']], id_vars = 'bsp', var_name = 'estimate')
    .assign(error = lambda x: abs(x['value'] - x['bsp']) / x['bsp'])
    .groupby('estimate', as_index=False)
    .agg({'error': 'mean'})
    .sort_values('error')
)

,estimate,error
6,rf_bsp_prediction,0.076963
0,back_best,0.084658
3,ladder_mid_point,0.086128
2,geometric_mid_point,0.087326
4,lay_best,0.098311
5,near_price,0.109640
1,far_price,0.501813
